In [ ]:
import json
import cv2
import numpy as np
import os
yourpath="C:/Users/Kompikl/Downloads/magisterka/magisterka/cessa"
coco_dict = {}
coco_dict['images'] = []
coco_dict['annotations'] = []
coco_dict['categories'] = [{'id': 1, 'name': 'bubbles'}]

image_id = 0
annotation_id = 0

for root, dirs, files in os.walk(yourpath, topdown=False):
    for name in files:
        if os.path.splitext(os.path.join(root, name))[1].lower() == ".jpg":
    # load image
            img = cv2.imread(os.path.join(root, name), 0)
            img = cv2.medianBlur(img,25)
            img = cv2.subtract(img, 30)
            circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT,
                    1.3, 40, param1=130, param2=50, minRadius=50, maxRadius=0)
            if circles is not None:
                circles = np.uint16(np.around(circles))

                    # add image to COCO dictionary
                image_info = {'id': image_id, 'license': 1, 'file_name': name, 'height': img.shape[0], 'width': img.shape[1], 'date_captured': '2023-05-04T07:18:25+00:00'}
                coco_dict['images'].append(image_info)

                # add annotations to COCO dictionary
                for c in circles[0, :]:
                        x, y, radius = c[0], c[1], c[2]
                        start_point = (int(x - radius), int(y - radius))
                        end_point = (int(x + radius), int(y + radius))
                        height = end_point[1] - start_point[1]
                        width = end_point[0] - start_point[0]
                        area = height * width

                        annotation_info = {'id': annotation_id, 'image_id': image_id, 'category_id': 1, 'bbox': [start_point[0], start_point[1], width, height], 'area': area, 'segmentation': [], 'iscrowd': 0}
                        coco_dict['annotations'].append(annotation_info)
                        annotation_id += 1

            image_id += 1
            with open('coco.json', 'w') as f:
                json.dump(coco_dict, f)